 # Import data from csv files


In [1]:
import numpy as np
import csv
import sys
from math import *

# import speed data
list_points_speed = []
with open('10-speed.csv','r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        points = ''.join(lines[i]).split(",")
        list_points_speed.append( [ float(points[j].replace('\x00', '')) for j in range(len(points)) ])
a = len(list_points_speed)
b = len(list_points_speed[0])
print(a,b)

1800 3600


In [2]:
# import probability data
list_points_proba = []
with open('10-probability.csv','r') as f:
    lines = f.readlines()
    for i in range(len(lines)):
        points = ''.join(lines[i]).split(",")
        list_points_proba.append( [ float(points[j].replace('\x00', '')) for j in range(len(points)) ])
print("Succes")

Succes


In [3]:
# generation of Neo4j nodes.
# indentify all no-empty cells
id_point = -1

#stock the id
ids_table = [[-1 for j in range(b)] for i in range(a)] 
ids_table2 = [[-1 for j in range(b)] for i in range(a)]

with open("tmp/new_points_10.txt", "a") as out1:
    for i in range(a):
        for j in range(b):
            id_point += 1
            ids_table2[i][j] = id_point
            out1.write("%d,%f,%f \n" %(id_point,-180+0.1*j,90-0.1*i))
            if(list_points_proba[i][j] != 0):
                ids_table[i][j] = 1
print("Succes")

Succes


In [4]:
speed_max = max(max(list_points_speed))
print(len(ids_table),len(ids_table[0]))

1800 3600


In [16]:
distance_geo(ids_table,900,901,0,1)

20000

In [15]:
# function calculating the geographical distance between given points
def distance_geo(ids_table,i1,i2,j1,j2):
    if( ids_table[i1][j1]==-1 or ids_table[i2][j2]==-1 ):
        return 20000
    else:
        lat1 = (90-0.1*i1)*pi/180
        lat2 = (90-0.1*i2)*pi/180
        lon1 = (180-0.1*j1)*pi/180
        lon2 = (180-0.1*j2)*pi/180
        d_lat = lat1 - lat2
        d_lon = lon1 - lon2
        a =  (sin(d_lat/2))**2 + cos(lat1) * cos(lat2) * ( sin(d_lon/2) )**2
        return 2*atan2( sqrt(a), sqrt(1-a))*6400/15.7   ##to normalize the distance

def distance_two_neighbourhood(tables, i1,j1,i2,j2, table_speed=list_points_speed, table_proba=list_points_proba,speed_max=speed_max):
    if( tables[i1][j1]==-1 or tables[i2][j2]==-1 ):
        return 20000
    else: 
        factor_speed =  1 - ( table_speed[i1][j1] + table_speed[i2][j2] ) / (2*speed_max) 
        factor_proba =  1 - ( table_proba[i1][j1] + table_proba[i2][j2] ) / 2 
        return distance_geo(ids_table, i1,i2,j1,j2) * factor_speed + factor_proba
    

In [17]:
# list of distances for each point
list_points = ids_table
list_distances = []

for i in range(a):
    list_distances.append([])
    for j in range(b):
        list_distances[i].append([])
        if ( i==0 and j==0 ):
            list_distances[i][j].extend([-1,
                                         -1,
                                         distance_two_neighbourhood(list_points,0,0,0,1),
                                         distance_two_neighbourhood(list_points,0,0,1,1),
                                         distance_two_neighbourhood(list_points,0,0,1,0),
                                         -1,
                                         -1,
                                         -1])
        
        if ( i !=0 and i!=a-1 and j==0 ):
            list_distances[i][j].extend([-1,
                                         distance_two_neighbourhood(list_points,i,j,i-1,j+1),
                                         distance_two_neighbourhood(list_points,i,j,i,  j+1),
                                         distance_two_neighbourhood(list_points,i,j,i+1,j+1),
                                         distance_two_neighbourhood(list_points,i,j,i+1,j),
                                         -1,
                                         -1,
                                         -1])
        if ( i==a-1 and j==0 ):
            list_distances[i][j].extend([-1,
                                         distance_two_neighbourhood(list_points,i,j,i-1,j+1),
                                         distance_two_neighbourhood(list_points,i,j,i,  j+1),
                                         -1,
                                         -1,
                                         -1,
                                         -1,
                                         -1])
        if ( i==0 and j!=0 and j!=b-1 ):
            list_distances[i][j].extend([-1,
                                         -1,
                                         distance_two_neighbourhood(list_points,i,j,i,  j+1),
                                         distance_two_neighbourhood(list_points,i,j,i+1,j+1),
                                         distance_two_neighbourhood(list_points,i,j,i+1,j),
                                         -1,
                                         -1,
                                         -1])
        if ( i==0 and j==b-1 ):
            list_distances[i][j].extend([-1,
                                         -1,
                                         distance_two_neighbourhood(list_points,i,j,i,0),
                                         distance_two_neighbourhood(list_points,i,j,i+1,0),
                                         distance_two_neighbourhood(list_points,i,j,i+1,j),
                                         -1,
                                         -1,
                                         -1])
        if ( i!=0 and i!=a-1 and j!=0 and j!=b-1 ):
            list_distances[i][j].extend([-1,
                                         distance_two_neighbourhood(list_points,i,j,i-1,j+1),
                                         distance_two_neighbourhood(list_points,i,j,i,  j+1),
                                         distance_two_neighbourhood(list_points,i,j,i+1,j+1),
                                         distance_two_neighbourhood(list_points,i,j,i+1,j),
                                         -1,
                                         -1,
                                         -1])
        if ( i!=0 and i!=a-1 and j==b-1):
            list_distances[i][j].extend([-1,
                                         distance_two_neighbourhood(list_points,i,j,i-1,0),
                                         distance_two_neighbourhood(list_points,i,j,i,0),
                                         distance_two_neighbourhood(list_points,i,j,i+1,0),
                                         distance_two_neighbourhood(list_points,i,j,i+1,j),
                                         -1,
                                         -1,
                                         -1])
        if ( i==a-1 and j==b-1):
            list_distances[i][j].extend([-1,
                                         distance_two_neighbourhood(list_points,i,j,i-1,0),
                                         distance_two_neighbourhood(list_points,i,j,i,0),
                                         -1,
                                         -1,
                                         -1,
                                         -1,
                                         -1])
        if ( i==a-1 and j!=0 and j!=b-1 ):
            list_distances[i][j].extend([-1,
                                         distance_two_neighbourhood(list_points,i,j,i-1,j+1),
                                         distance_two_neighbourhood(list_points,i,j,i,  j+1),
                                         -1,
                                         -1,
                                         -1,
                                         -1,
                                         -1])
print("ok")

ok


In [19]:
# generation of Neo4j relations(arc)
ids_table = ids_table2
with open("tmp/new_relations_10.txt", "a") as out2:
    for i in range(a):
        for j in range(b):
            for k in range(8):
                if ( list_distances[i][j][k]!=-1 and j==0 ):
                    if k==1:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i-1][j+1],list_distances[i][j][k]))
                    if k==2:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i][j+1],list_distances[i][j][k]))
                    if k==3:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i+1][j+1],list_distances[i][j][k]))
                    if k==4:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i+1][j],list_distances[i][j][k]))
                
                if ( list_distances[i][j][k]!=-1 and j==b-1 ):
                    if k==1:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i-1][0],list_distances[i][j][k]))
                    if k==2:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i][0],list_distances[i][j][k]))
                    if k==3:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i+1][0],list_distances[i][j][k]))
                    if k==4:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i+1][j],list_distances[i][j][k]))
                if ( list_distances[i][j][k] != -1 and j!=0 and j!=b-1 ):
                    if k==1:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i-1][j+1],list_distances[i][j][k]))
                    if k==2:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i][j+1],list_distances[i][j][k]))
                    if k==3:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i+1][j+1],list_distances[i][j][k]))
                    if k==4:
                        out2.write("%d,%d,%f \n" %(ids_table[i][j],ids_table[i+1][j],list_distances[i][j][k]))

In [64]:
list_distances[428][3599]

[-1,
 2.7720897955867834,
 2.3955918206871063,
 2.6604539753286023,
 2.5328542905664624,
 -1,
 -1,
 -1]